In [1]:
#Instalación de librerías python necesarias
%pip install requests
%pip install beautifulsoup4
%pip install pandas


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#Importar librerías para extracción de datos web
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
response = requests.get('https://scrapepark.org/')
print(response.status_code)
#print(response.text)

200


In [4]:
#Sitio web de la extracción de datos
url         = "https://scrapepark.org/"
response    = requests.get(url)
soup        = BeautifulSoup(response.content, 'html.parser')

In [5]:
#ejemplo de lectura y extracción de un html con la librería BeautifulSoup
html_doc    = "<html><head><title>Example</title></head><body><p>Bienvenidos</p></body></html>"
soup        = BeautifulSoup(html_doc, 'html.parser')
print(soup.body.string)

Bienvenidos


In [6]:
#Sitio web de la extracción de datos
url = "http://books.toscrape.com/"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
#recorriendo los datos y almacenandolos en una lista
books = soup.find_all('article', class_='product_pod') # accede a la etiqueta article y la clase product_pod
book_list = [] #crea una lista vacia

for book in books: #creando estructura repetitiva for
    title = book.h3.a['title']
    price = book.find('p', class_='price_color').text
    book_list.append({"Title": title, "Price": price})


In [ ]:
df_books = pd.DataFrame(book_list)
print(df_books.head())

                                   Title   Price
0                   A Light in the Attic  £51.77
1                     Tipping the Velvet  £53.74
2                             Soumission  £50.10
3                          Sharp Objects  £47.82
4  Sapiens: A Brief History of Humankind  £54.23


In [ ]:
# convierte todos los valores de la columna Price a dolares
# Tasa de cambio (1 GBP = 1.27 USD)
exchange_rate = 1.27

df_books['Price'] = (
    df_books['Price']
    .astype(str)  # Convierte a cadena
    .str.replace('£', '', regex=False)  # Quita el símbolo de libra
    .str.strip()  # Limpia espacios
    .replace('', None)  # Maneja valores vacíos
    .astype(float)  # Convierte a float
)

# Convierte a dólares
df_books['Price (USD)'] = df_books['Price'] * exchange_rate

In [ ]:
df_books.head()

,Title,Price,Price (USD)
0,A Light in the Attic,51.77,65.7479
1,Tipping the Velvet,53.74,68.2498
2,Soumission,50.10,63.6270
3,Sharp Objects,47.82,60.7314
4,Sapiens: A Brief History of Humankind,54.23,68.8721


In [12]:
df_books_sel = df_books[['Title', 'Price (USD)']]
print(df_books_sel.head())

                                   Title  Price (USD)
0                   A Light in the Attic      65.7479
1                     Tipping the Velvet      68.2498
2                             Soumission      63.6270
3                          Sharp Objects      60.7314
4  Sapiens: A Brief History of Humankind      68.8721


In [13]:
#Se realiza visualización de los datos con la librería plotly
import plotly.express as px

fig = px.bar(df_books, x='Title', y='Price (USD)', title='Book Prices')
fig.show()

In [22]:
import plotly.express as px

# Obtener los 10 libros más caros
top_expensive_books = df_books.nlargest(10, 'Price')

# Crear el gráfico de barras horizontal
fig = px.bar(
    top_expensive_books,
    x='Price',
    y='Title',
    orientation='h',
    title='Top 10 Libros Más Caros',
    labels={'Price': 'Precio (USD)', 'Title': 'Título'},
    color='Price',  # Para variar el color según el precio
    color_continuous_scale='viridis'  # Escala de colores
)

# Mejorar el diseño
fig.update_layout(
    xaxis=dict(title='Precio (USD)'),
    yaxis=dict(title='Título', categoryorder='total ascending'),
    template='plotly_white',  # Fondo blanco y limpio
    title_font_size=20,
    title_x=0.5  # Centrar el título
)

# Mostrar el gráfico
fig.show()


In [21]:
import plotly.express as px
# Crear gráfico de rango de precios (histograma)
fig2 = px.histogram(
    df_books,
    x='Price',
    nbins=20,
    title='Distribución de Precios de los Libros',
    labels={'Price': 'Precio'},
    color='Price',
)

fig2.update_layout(
    xaxis=dict(title='Precio', range=[0, df_books['Price'].max()]),
    yaxis=dict(title='Frecuencia'),
    template='plotly_white',
    title_font_size=20,
    title_x=0.5
)

# Mostrar gráfico de distribución de precios
fig2.show()

# Resumen estadístico
print('Resumen Estadistico de los precios')
summary_stats = df_books['Price'].describe()
print(summary_stats)


Resumen Estadistico de los precios
count    20.000000
mean     38.048500
std      15.135231
min      13.990000
25%      22.637500
50%      41.380000
75%      51.865000
max      57.250000
Name: Price, dtype: float64
